## LCEL

### Traditional LLMCHain

In [1]:
from langchain import PromptTemplate

prompt_template = "Give me a small report on {topic}"

prompt = PromptTemplate(
    input_variables=['topic'],
    template=prompt_template
)

In [3]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen3:0.6b", temperature=0.0)

In [4]:
llm.invoke("Hello there")

AIMessage(content='<think>\nOkay, the user said "Hello there." I need to respond appropriately. Let me start by acknowledging their greeting. A simple "Hello there!" is a good start. Then, I should offer assistance. Maybe ask how I can help them. It\'s important to be friendly and open-ended. Let me make sure the response is natural and conversational. I should avoid any technical terms and keep the tone friendly. Alright, putting it all together.\n</think>\n\nHello there! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'model': 'qwen3:0.6b', 'created_at': '2025-05-18T21:56:07.536755542Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2337117464, 'load_duration': 1736354519, 'prompt_eval_count': 10, 'prompt_eval_duration': 221821656, 'eval_count': 107, 'eval_duration': 378112410, 'model_name': 'qwen3:0.6b'}, id='run--2289ed1c-c0fa-4e47-9a66-5ba87aa8263f-0', usage_metadata={'input_tokens': 10, 'output_tokens': 107, 'total_tokens': 117})

In [5]:
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=prompt,
    llm=llm,
    output_parser=output_parser
)

/tmp/ipykernel_139388/841205956.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [7]:
chain.invoke("AI")

{'topic': 'AI',
 'text': '<think>\nOkay, the user wants a small report on AI. Let me start by understanding the context. They might be a student, a researcher, or someone interested in AI. Since it\'s a small report, I should keep it concise but informative.\n\nFirst, I need to outline the main points. Maybe start with an introduction explaining what AI is and its importance. Then, discuss the key areas: machine learning, natural language processing, computer vision, and robotics. Each section should have a brief explanation and maybe a brief example. Including some statistics or recent developments would add value. Finally, a conclusion to summarize the main points.\n\nWait, the user mentioned "small report," so I should avoid too much detail. Make sure each section is concise. Also, check for any technical terms and explain them if necessary. Ensure the report flows well and is easy to read. Let me structure it step by step to cover all necessary aspects without overwhelming the read

This is quite restrictive and has a lot of requirements to be pre initialized

### LCEL Approach

In [8]:
lcel_chain = prompt | llm | output_parser

In [9]:
lcel_chain.invoke("RAG")

'<think>\nOkay, the user wants a small report on RAG. Let me start by understanding what RAG is. RAG stands for Retrieval-Augmented Generation, which is a technique used in AI to improve the performance of large language models. It combines the strengths of both retrieval and generation models.\n\nFirst, I need to outline the main components of RAG. Retrieval is about finding relevant information, and generation is about creating new text. So, the report should explain both parts. Maybe start with the basics, like how RAG works. Then, discuss the benefits, such as improved accuracy and efficiency. \n\nI should also mention the applications, like in research papers, news articles, or customer service. That shows real-world use. Including some examples would make it more concrete. Maybe mention specific industries where RAG is applied, like healthcare or finance. \n\nWait, the user said "small report," so it should be concise. Avoid too much technical jargon. Make sure to highlight the k

How does LCEL work?

The concept is reasonably simple, you start on the far left side of the line, look at the first variable, and the output to that variable is passed into the next variable, before we had prompt | llm | output_parser, we can see that the prompt, feeds into the model, then the model result feeds into the output parser.

When we use the pipe operator | we are basically looking for a or function, this is where we can find a chained functionallity to the variable.

In [ ]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # Run the other function with its inputs as the output of the Runnable function
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

To create such Runnable class in LangChain, we use RunnableLambda

In [11]:
from langchain_core.runnables import RunnableLambda

def add_five(x):
    return x+5
def sub_five(x):
    return x-5
def mul_five(x):
    return x*5

add_five = RunnableLambda(add_five)
sub_five = RunnableLambda(sub_five)
mul_five = RunnableLambda(mul_five)

chain = add_five | sub_five | mul_five

In [12]:
chain.invoke(3)

15